In [1]:
#!pip install streamlit plotly

In [2]:
#!pip install -U plotly

In [3]:
import datetime
import time # para medir tiempo
import math

In [4]:
# Tiempo inicial
start = time.time()  

In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta, time as dtime, timezone
import time  # para medir tiempo
import math
import warnings
from urllib3.exceptions import InsecureRequestWarning

warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Estaciones de precipitación (sp)
sp_codes = ["101", "102", "103", "104", "106", "108", "109", "131", "132", "133", "134", "135", 
            "136", "137", "138", "139", "140", "141", "142", "143", "144", "145", "146", "147", 
            "149", "150", "151", "152", "154", "155", "156", "157","158","159", "160", "161", "162", "163"]

# Estaciones de precipitación (sp)
#sp_codes = ["149", "150", "151", "152", "154", "155", "156", "157"]

def obtener_datos_estacion(code, calidad=1):
    page = 1
    datos = []
    while True:
        url = f"https://sigran.antioquia.gov.co/api/v1/estaciones/sp_{code}/precipitacion?calidad={calidad}&page={page}"
        response = requests.get(url, verify=False)
        if response.status_code != 200:
            break
        data = response.json()
        values = data.get("values", [])
        if not values:
            break
        datos.extend(values)
        page += 1
        # Paramos si ya tenemos más de 72 horas de datos
        fechas = [pd.to_datetime(d['fecha']) for d in datos]
        if fechas and (max(fechas) - min(fechas)).total_seconds() > 72 * 3600:
            break
    return datos



In [6]:
def obtener_metadata_sp(code):
    url = f"https://sigran.antioquia.gov.co/api/v1/estaciones/sp_{code}/"
    resp = requests.get(url, verify=False)
    if resp.status_code == 200:
        d = resp.json()
        return {
            "estacion": code,
            "codigo": d.get("codigo"),
            "descripcion": d.get("descripcion"),
            "nombre_web": d.get("nombre_web"),
            "latitud": float(d.get("latitud", 0)),
            "longitud": float(d.get("longitud", 0)),
            "municipio": d.get("municipio"),
            "region": d.get("region")
        }
    else:
        return None

In [7]:
import pytz

def procesar_datos(datos, ahora=None):
    if not datos:
        return None

    df = pd.DataFrame(datos)
    df["fecha"] = pd.to_datetime(df["fecha"], utc=True)
    df["muestra"] = pd.to_numeric(df["muestra"], errors='coerce')

    ahora = ahora or datetime.utcnow().replace(tzinfo=pytz.UTC)

    acumulados = {
        "acum_6h": df[(df["fecha"] > ahora - timedelta(hours=6)) & (df["fecha"] <= ahora)]["muestra"].sum(),
        "acum_24h": df[(df["fecha"] > ahora - timedelta(hours=24)) & (df["fecha"] <= ahora)]["muestra"].sum(),
        "acum_72h": df[(df["fecha"] > ahora - timedelta(hours=72)) & (df["fecha"] <= ahora)]["muestra"].sum()
    }

    # Serie de 120 horas
    serie_120h = []
    for h in range(1, 121):
        t_ini = ahora - timedelta(hours=h)
        t_fin = ahora - timedelta(hours=h-1)
        val = df[(df["fecha"] > t_ini) & (df["fecha"] <= t_fin)]["muestra"].sum()
        serie_120h.append({"hora": t_ini, "acumulado": val})

    # Día meteorológico: 7am local = 12:00 UTC del día anterior
    ult_utc = ahora.replace(minute=0, second=0, microsecond=0)
    if ult_utc.hour < 12:
        dia_base = ult_utc - timedelta(days=1)
    else:
        dia_base = ult_utc
    inicio_dia = dia_base.replace(hour=12)
    
    #def acum_dias(n):
        #return df[(df["fecha"] > inicio_dia - timedelta(days=n)) & (df["fecha"] <= inicio_dia)]["muestra"].sum()
    
    #meteo = {
        #"ultimo_dia_meteorologico": acum_dias(1),
        #"ultimos_7_dias_meteorologicos": acum_dias(7),
        #"ultimos_30_dias_meteorologicos": acum_dias(30)
    #}

    def acum_dias_meteorologicos(n, df):
        # Momento actual en UTC
        ahora = datetime.now(timezone.utc)

        # 7:00 AM UTC del día actual (inicio del día meteorológico actual)
        inicio_meteo = datetime.combine(ahora.date(), dtime(7, 0, tzinfo=timezone.utc))


        # Rango del día meteorológico: de (hace n días a las 7 AM) hasta (hoy a las 7 AM)
        fecha_inicio = inicio_meteo - timedelta(days=n)
        fecha_fin = inicio_meteo

        # Filtrar y sumar la columna 'muestra' en ese rango
        return df[(df["fecha"] > fecha_inicio) & (df["fecha"] <= fecha_fin)]["muestra"].sum()

    # Diccionario con los acumulados meteorológicos
    meteo = {
        "ultimo_dia_meteorologico": acum_dias_meteorologicos(1, df),
        "ultimos_7_dias_meteorologicos": acum_dias_meteorologicos(7, df),
        "ultimos_30_dias_meteorologicos": acum_dias_meteorologicos(30, df)
    }

    
    fecha_max = df["fecha"].max()
    dias_sin_datos = (ahora - fecha_max).days
    datos_recientes = int((ahora - fecha_max) <= timedelta(days=1))
    

    return {
        **acumulados,
        **meteo,
        "datos_recientes": datos_recientes,
        "dias_sin_datos": dias_sin_datos,
        "fecha_ultimo_dato": fecha_max, 
        "serie_120h": serie_120h
    }


In [8]:
resultados = []
metadata = []

for code in sp_codes:
    datos = obtener_datos_estacion(code)
    resumen = procesar_datos(datos)
    meta = obtener_metadata_sp(code)
    if resumen and meta:
        resumen["estacion"] = code
        meta.update(resumen)
        resultados.append(resumen)
        metadata.append(meta)

df_meta = pd.DataFrame(metadata)

/var/folders/8c/s98grqnx6v56_mmc7xz85wq40000gn/T/ipykernel_55149/1942307016.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ahora = ahora or datetime.utcnow().replace(tzinfo=pytz.UTC)
/var/folders/8c/s98grqnx6v56_mmc7xz85wq40000gn/T/ipykernel_55149/1942307016.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ahora = ahora or datetime.utcnow().replace(tzinfo=pytz.UTC)
/var/folders/8c/s98grqnx6v56_mmc7xz85wq40000gn/T/ipykernel_55149/1942307016.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ahora = ahora or

In [9]:
df_meta

,estacion,codigo,descripcion,nombre_web,latitud,longitud,municipio,region,acum_6h,acum_24h,acum_72h,ultimo_dia_meteorologico,ultimos_7_dias_meteorologicos,ultimos_30_dias_meteorologicos,datos_recientes,dias_sin_datos,fecha_ultimo_dato,serie_120h
0,101,sp_101,"Latitud 6.11384, longitud -75.98488","sp_101 - Latitud 6.11384, longitud -75.98488",6.112790,-75.990630,89,8,0.0,31.242,33.274,31.242,33.274,33.274,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:06.683051+00:00, 'a..."
1,102,sp_102,"Latitud 7.75208, longitud -75.25359","sp_102 - Latitud 7.75208, longitud -75.25359",7.752100,-75.253580,117,2,0.0,22.352,33.782,22.352,64.008,64.008,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:07.397563+00:00, 'a..."
2,103,sp_103,latitud 5.99233 longitud -7604976,sp_103 - latitud 5.99233 longitud -7604976,5.992330,-76.049760,85,8,0.0,44.450,44.958,44.450,44.958,44.958,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:08.071103+00:00, 'a..."
3,104,sp_104,latitud 6.90747 longitud -756721,sp_104 - latitud 6.90747 longitud -756721,6.907470,-75.672100,49,5,0.0,0.000,0.254,0.254,0.762,0.762,0,1,2025-09-01 14:31:00+00:00,"[{'hora': 2025-09-02 15:16:08.794983+00:00, 'a..."
4,106,sp_106,Latitud 5.71657 longitud -74.81221666666667,sp_106 - Latitud 5.71657 longitud -74.81221666...,5.716550,-74.812200,18,7,0.0,0.000,0.000,0.000,1.200,1.200,0,4,2025-08-29 10:17:00+00:00,"[{'hora': 2025-09-02 15:16:09.450772+00:00, 'a..."
5,108,sp_108,Latitud 7.43319 longitud -74.86089,sp_108 - Latitud 7.43319 longitud -74.86089,7.433190,-74.860890,113,2,0.0,0.000,0.000,0.000,0.000,39.624,0,11,2025-08-22 01:01:00+00:00,"[{'hora': 2025-09-02 15:16:10.154604+00:00, 'a..."
6,109,sp_109,"Longitud -76.51716, Latitud 7.29571","sp_109 - Longitud -76.51716, Latitud 7.29571",7.295690,-76.517340,58,9,0.0,31.242,129.540,54.356,132.080,132.080,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:10.943807+00:00, 'a..."
7,131,sp_131,"longitud 5.6478123 latitud -75,9518005","sp_131 - longitud 5.6478123 latitud -75,9518005",5.647880,-75.951490,71,8,0.0,2.794,28.194,2.794,28.702,28.702,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:11.681404+00:00, 'a..."
8,132,sp_132,"longitud 8.6150 latitud -76,3817","sp_132 - longitud 8.6150 latitud -76,3817",8.614980,-76.381780,125,9,0.0,2.286,3.302,2.032,6.096,6.096,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:12.442544+00:00, 'a..."
9,133,sp_133,"longitud 6,5596 latitud -75,1033","sp_133 - longitud 6,5596 latitud -75,1033",6.559650,-75.103290,104,4,0.0,57.912,58.166,57.912,58.420,58.420,1,0,2025-09-02 11:16:00+00:00,"[{'hora': 2025-09-02 15:16:13.176204+00:00, 'a..."


In [10]:
df_meta.columns

Index(['estacion', 'codigo', 'descripcion', 'nombre_web', 'latitud',
       'longitud', 'municipio', 'region', 'acum_6h', 'acum_24h', 'acum_72h',
       'ultimo_dia_meteorologico', 'ultimos_7_dias_meteorologicos',
       'ultimos_30_dias_meteorologicos', 'datos_recientes', 'dias_sin_datos',
       'fecha_ultimo_dato', 'serie_120h'],
      dtype='object')

In [11]:
df_meta.to_csv("metadatos_estaciones_precipitacion.csv", index=False)

In [12]:
# Convertir a DataFrame para resumen (sin la serie de 120h)
df_resultado = pd.DataFrame([{k: v for k, v in r.items() if k != "serie_120h"} for r in resultados])

#print(df_resultado.head())

In [13]:
# Ordenar por defecto: estaciones con datos recientes primero, y por fecha más reciente
df_resultado = df_resultado.sort_values(by=["datos_recientes", "fecha_ultimo_dato"], ascending=[False, False])

In [14]:
# Crear copia con etiquetas legibles
df_pie = df_resultado.copy()
df_pie['datos_recientes'] = df_pie['datos_recientes'].map({1: 'Reciente', 0: 'No reciente'})

In [15]:
# Registros con datos recientes (menos de 7 días sin datos)
df_reciente = df_resultado[df_resultado["dias_sin_datos"] < 7].copy()
df_reciente = df_reciente.sort_values(by='estacion', ascending=True)


# Registros sin datos recientes (7 días o más sin datos)
df_no_reciente = df_resultado[df_resultado["dias_sin_datos"] >= 7].copy()

In [16]:
#print(f"Con datos recientes: {len(df_reciente)} estaciones")
#print(f"Sin datos recientes: {len(df_no_reciente)} estaciones")


### Cruce de Info API con datos de Municipio y Subregión

In [17]:
# Cargar el archivo Excel (Base de datos estaciones SAMA)
df_excel = pd.read_excel('Base de datos estaciones SAMA.xlsx', usecols=[
    'GRUPO', 'MUNICIPIO', 'NOM_EST', 'COD_EST', 'TIPO', 'COMUN_PRIORIZ', 'CORRIENTE', 'LAT', 'LONG'
])

# Reorganizar las columnas como se indicó
df_excel = df_excel[['COD_EST', 'TIPO', 'GRUPO', 'MUNICIPIO', 'NOM_EST', 'COMUN_PRIORIZ', 'CORRIENTE', 'LAT', 'LONG']]

# LIMPIAR columna COD_EST
df_excel['COD_EST'] = df_excel['COD_EST'].astype(str).str.strip().str.lower()
#df_excel

### Correción de regiones erroneas

In [18]:
# Diccionario con los valores correctos
correcciones = {
    'sp_163': 8,
    'sp_149': 3,
    'sp_151': 6,
    'sp_158': 6
}

# Aplicar las correcciones
for codigo, region_correcta in correcciones.items():
    df_meta.loc[df_meta['codigo'] == codigo, 'region'] = region_correcta


In [19]:
# 1. Renombrar la columna 'municipio' en df_meta
df_meta = df_meta.rename(columns={'municipio': 'municipio_num'})

# 2. Crear un DataFrame auxiliar con solo las columnas necesarias de df_excel
df_municipio = df_excel[['COD_EST', 'MUNICIPIO']].rename(columns={
    'COD_EST': 'codigo',  # para que coincida con df_meta
    'MUNICIPIO': 'municipio'
})

# 3. Hacer el merge con df_meta usando la columna común 'codigo'
df_meta = df_meta.merge(df_municipio, on='codigo', how='left')

df_meta['municipio'] = df_meta['municipio'].str.capitalize()

In [20]:
df_meta.loc[df_meta['codigo'] == 'sp_151', 'municipio'] = 'Sonson'

In [21]:
#Renombrar la columna 'region' a 'subregion_num'
df_meta = df_meta.rename(columns={'region': 'subregion_num'})

# Diccionario de equivalencias de subregiones
mapa_subregiones = {
    1: 'Valle de Aburra',
    2: 'Bajo Cauca',
    3: 'Magdalena Medio',
    4: 'Nordeste',
    5: 'Norte',
    6: 'Oriente',
    7: 'Occidente',
    8: 'Suroeste',
    9: 'Urabá'
}

# Creación de la columna 'subregion' usando el diccionario
df_meta['subregion'] = df_meta['subregion_num'].map(mapa_subregiones)


In [22]:
#df_meta[df_meta['municipio'].isna()]

In [23]:
#df_meta[df_meta['subregion'].isna()]

## Para tablero

In [24]:
# Instalar/reinstalar Streamlit en el kernel actual
import subprocess
import sys

try:
    import streamlit
    print("✅ Streamlit ya está disponible")
except ImportError:
    print("⚠️ Instalando Streamlit...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit"])
    print("✅ Streamlit instalado exitosamente")
    
try:
    import pytz
    print("✅ pytz ya está disponible")
except ImportError:
    print("⚠️ Instalando pytz...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pytz"])
    print("✅ pytz instalado exitosamente")

print("🎉 Todas las dependencias están listas!")

✅ Streamlit ya está disponible
✅ pytz ya está disponible
🎉 Todas las dependencias están listas!


In [25]:
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta

# Configuración de la página
st.set_page_config(
    page_title="🌧️ Tablero de estaciones de precipitación",
    page_icon="🌧️",
    layout="wide"
)

st.title("🌧️ Tablero de estaciones de precipitación")

# Sidebar con filtros
st.sidebar.header("Filtros")

# Filtro por subregión
subregiones = sorted(df_meta["subregion"].unique())
subregion_seleccionada = st.sidebar.selectbox(
    "Filtrar por subregión:",
    options=["Todas"] + subregiones,
    index=0
)

# Filtro por municipio (dinámico basado en subregión)
if subregion_seleccionada != "Todas":
    municipios = sorted(df_meta[df_meta["subregion"] == subregion_seleccionada]["municipio"].dropna().unique())
else:
    municipios = sorted(df_meta["municipio"].dropna().unique())

municipio_seleccionado = st.sidebar.selectbox(
    "Filtrar por municipio:",
    options=["Todos"] + municipios,
    index=0
)

# Filtro de estaciones (dinámico basado en filtros anteriores)
df_filtrado = df_meta.copy()
if subregion_seleccionada != "Todas":
    df_filtrado = df_filtrado[df_filtrado["subregion"] == subregion_seleccionada]
if municipio_seleccionado != "Todos":
    df_filtrado = df_filtrado[df_filtrado["municipio"] == municipio_seleccionado]

estaciones = sorted(df_filtrado["estacion"].unique())
estacion_seleccionada = st.sidebar.selectbox(
    "Selecciona estación:",
    options=[f"sp_{e}" for e in estaciones],
    index=0 if estaciones else None
)

# Crear columnas para el layout
col1, col2 = st.columns(2)

# Serie de tiempo de 120 horas
with col1:
    st.subheader("Serie de tiempo 120h")
    if estacion_seleccionada:
        estacion_id = estacion_seleccionada.replace("sp_", "")
        serie = next((r["serie_120h"] for r in resultados if r["estacion"] == estacion_id), [])
        if serie:
            df_serie = pd.DataFrame(serie)
            fig_serie = px.line(df_serie, x="hora", y="acumulado", 
                              title=f"Serie 120h - {estacion_seleccionada}")
            fig_serie.update_layout(xaxis_title="Hora", yaxis_title="Acumulado (mm)")
            st.plotly_chart(fig_serie, use_container_width=True)
        else:
            st.info("No hay datos disponibles para esta estación")

# Mapa de ubicación
with col2:
    st.subheader("📍 Ubicación de la estación")
    if estacion_seleccionada:
        estacion_id = estacion_seleccionada.replace("sp_", "")
        fila = df_meta[df_meta["estacion"] == estacion_id]
        if not fila.empty:
            fig_map = px.scatter_map(
                fila,
                lat="latitud",
                lon="longitud",
                hover_name="estacion",
                hover_data=["municipio", "subregion"],
                color_discrete_sequence=["red"],
                zoom=10,
                height=400
            )
            fig_map.update_layout(
                mapbox_style="carto-positron",
                margin={"r": 0, "t": 0, "l": 0, "b": 0},
                showlegend=False
            )
            st.plotly_chart(fig_map, use_container_width=True)
        else:
            st.info("Ubicación no disponible")

# Preparar datos para las tablas con filtros aplicados
df_tabla = df_reciente.merge(df_meta[["estacion", "subregion", "municipio"]], on="estacion", how="left")
if subregion_seleccionada != "Todas":
    df_tabla = df_tabla[df_tabla["subregion"] == subregion_seleccionada]
if municipio_seleccionado != "Todos":
    df_tabla = df_tabla[df_tabla["municipio"] == municipio_seleccionado]

# Tabla de acumulados recientes
st.subheader("Acumulados recientes por estación")
if not df_tabla.empty:
    df_acumulados = df_tabla[["estacion", "acum_6h", "acum_24h", "acum_72h"]].copy()
    df_acumulados["estacion"] = df_acumulados["estacion"].apply(lambda x: f"sp_{x}")
    df_acumulados = df_acumulados.round(3)
    
    col1, col2 = st.columns([3, 1])
    with col1:
        st.dataframe(df_acumulados, use_container_width=True)
    with col2:
        csv_acumulados = df_acumulados.to_csv(index=False)
        st.download_button(
            label="Descargar CSV",
            data=csv_acumulados,
            file_name="acumulados_estaciones.csv",
            mime="text/csv"
        )

# Tabla de acumulados meteorológicos
st.subheader("Acumulados meteorológicos por estación")
if not df_tabla.empty:
    df_meteo = df_tabla[["estacion", "ultimo_dia_meteorologico", "ultimos_7_dias_meteorologicos", "ultimos_30_dias_meteorologicos"]].copy()
    df_meteo["estacion"] = df_meteo["estacion"].apply(lambda x: f"sp_{x}")
    df_meteo = df_meteo.round(3)
    
    col1, col2 = st.columns([3, 1])
    with col1:
        st.dataframe(df_meteo, use_container_width=True)
    with col2:
        csv_meteo = df_meteo.to_csv(index=False)
        st.download_button(
            label="Descargar CSV",
            data=csv_meteo,
            file_name="acumulados_meteorologicos.csv",
            mime="text/csv"
        )

# Gráfico de acumulados meteorológicos del último día
st.subheader("Acumulado meteorológico del último día por estación")
if not df_tabla.empty:
    df_grafico = df_tabla.copy()
    df_grafico["estacion"] = df_grafico["estacion"].apply(lambda x: f"sp_{x}")
    
    fig_meteo = px.bar(
        df_grafico,
        x="estacion",
        y="ultimo_dia_meteorologico",
        title="Acumulado del último día meteorológico"
    )
    fig_meteo.update_layout(
        xaxis_title="Estación",
        yaxis_title="Acumulado (mm)",
        showlegend=False,
        xaxis_tickangle=45
    )
    st.plotly_chart(fig_meteo, use_container_width=True)

# Gráfico de estaciones sin datos
st.subheader("Estaciones sin datos por más de 7 días")
if not df_no_reciente.empty:
    df_sin_datos = df_no_reciente.copy()
    df_sin_datos["estacion"] = df_sin_datos["estacion"].apply(lambda x: f"sp_{x}")
    df_sin_datos = df_sin_datos.sort_values("dias_sin_datos", ascending=False)
    
    fig_sin_datos = px.bar(
        df_sin_datos,
        x="estacion", 
        y="dias_sin_datos",
        title="Días sin datos por estación"
    )
    fig_sin_datos.update_layout(xaxis_tickangle=45)
    st.plotly_chart(fig_sin_datos, use_container_width=True)

# Gráfico de disponibilidad de datos
st.subheader("Disponibilidad de datos recientes")
df_pie_chart = df_pie['datos_recientes'].value_counts()
total_estaciones = len(df_pie)

porcentajes = []
labels = []
values = []

for label in df_pie_chart.index:
    count = df_pie_chart[label]
    percentage = (count / total_estaciones) * 100
    porcentajes.append(f"{percentage:.1f}% ({count})")
    labels.append(label)
    values.append(count)

fig_disponibilidad = go.Figure(data=[
    go.Bar(
        x=labels,
        y=[(v/total_estaciones)*100 for v in values],
        text=porcentajes,
        textposition='inside',
        marker_color=['lightcoral' if 'No' in label else 'lightblue' for label in labels]
    )
])

fig_disponibilidad.update_layout(
    title="Porcentaje de estaciones con/sin datos recientes",
    xaxis_title="Estado de los datos",
    yaxis_title="Porcentaje (%)",
    yaxis=dict(range=[0, 100]),
    height=400
)

st.plotly_chart(fig_disponibilidad, use_container_width=True)

2025-09-02 11:16:32.969 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 11:16:32.969 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 11:16:33.030 
  command:

    streamlit run /Users/sergiocamilogarzonperez/Projects/sama/pronosticosnb/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-02 11:16:33.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 11:16:33.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 11:16:33.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 11:16:33.031 Thread 'MainThread': missing Scri

DeltaGenerator()

## 📊 Visualizaciones para el Notebook (sin Streamlit)

Las siguientes celdas muestran las gráficas usando solo Plotly, que funciona perfectamente en Jupyter:

In [26]:
# Gráficas que SÍ funcionan en el notebook (solo Plotly)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print(f"📊 Total de estaciones procesadas: {len(df_meta)}")
print(f"✅ Estaciones con datos recientes: {len(df_reciente)}")
print(f"⚠️ Estaciones sin datos recientes: {len(df_no_reciente)}")

# 1. Gráfico de disponibilidad de datos
df_pie_chart = df_pie['datos_recientes'].value_counts()
total_estaciones = len(df_pie)

fig_disponibilidad = px.pie(
    values=df_pie_chart.values, 
    names=df_pie_chart.index,
    title="Disponibilidad de datos recientes por estación",
    color_discrete_map={'Reciente': 'lightblue', 'No reciente': 'lightcoral'}
)

# Mostrar la gráfica
fig_disponibilidad

# 2. Gráfico de estaciones sin datos (si hay alguna)
if not df_no_reciente.empty:
    df_sin_datos = df_no_reciente.copy()
    df_sin_datos["estacion"] = df_sin_datos["estacion"].apply(lambda x: f"sp_{x}")
    df_sin_datos = df_sin_datos.sort_values("dias_sin_datos", ascending=False)
    
    fig_sin_datos = px.bar(
        df_sin_datos,
        x="estacion", 
        y="dias_sin_datos",
        title="Estaciones sin datos por más de 7 días",
        labels={'dias_sin_datos': 'Días sin datos', 'estacion': 'Estación'}
    )
    fig_sin_datos.update_layout(xaxis_tickangle=45)
    
    print(f"\n📊 Hay {len(df_sin_datos)} estaciones sin datos recientes:")
    for _, row in df_sin_datos.iterrows():
        print(f"  - {row['estacion']}: {row['dias_sin_datos']} días sin datos")
else:
    print("🎉 ¡Todas las estaciones tienen datos recientes!")

📊 Total de estaciones procesadas: 36
✅ Estaciones con datos recientes: 34
⚠️ Estaciones sin datos recientes: 2

📊 Hay 2 estaciones sin datos recientes:
  - sp_147: 298 días sin datos
  - sp_108: 11 días sin datos


In [27]:
# Simplificado: Análisis de datos sin gráficas complejas

print("📊 RESUMEN DE ESTACIONES DE PRECIPITACIÓN")
print("=" * 50)

# Análisis básico
print(f"📍 Total de estaciones: {len(df_meta)}")
print(f"✅ Con datos recientes: {len(df_reciente)}")
print(f"⚠️ Sin datos recientes: {len(df_no_reciente)}")
print(f"📈 Porcentaje operativo: {(len(df_reciente)/len(df_meta)*100):.1f}%")

print("\n🏙️ DISTRIBUCIÓN POR SUBREGIÓN:")
subregion_count = df_meta['subregion'].value_counts()
for subregion, count in subregion_count.items():
    print(f"  - {subregion}: {count} estaciones")

print("\n💧 TOP 10 ESTACIONES - ÚLTIMO DÍA METEOROLÓGICO:")
top_acum = df_reciente.nlargest(10, 'ultimo_dia_meteorologico')[['estacion', 'ultimo_dia_meteorologico']]
for _, row in top_acum.iterrows():
    print(f"  - sp_{row['estacion']}: {row['ultimo_dia_meteorologico']:.2f} mm")

print("\n⏰ ACUMULADOS PROMEDIO:")
print(f"  - 6 horas: {df_reciente['acum_6h'].mean():.2f} mm")
print(f"  - 24 horas: {df_reciente['acum_24h'].mean():.2f} mm") 
print(f"  - 72 horas: {df_reciente['acum_72h'].mean():.2f} mm")

if not df_no_reciente.empty:
    print(f"\n❌ ESTACIONES PROBLEMÁTICAS:")
    for _, row in df_no_reciente.iterrows():
        print(f"  - sp_{row['estacion']}: {row['dias_sin_datos']} días sin datos")

print(f"\n📄 Archivo CSV generado: metadatos_estaciones_precipitacion.csv")
print(f"🌐 Aplicación Streamlit: streamlit_app.py")
print(f"⚡ Para gráficas interactivas, ejecuta: streamlit run streamlit_app.py")

📊 RESUMEN DE ESTACIONES DE PRECIPITACIÓN
📍 Total de estaciones: 36
✅ Con datos recientes: 34
⚠️ Sin datos recientes: 2
📈 Porcentaje operativo: 94.4%

🏙️ DISTRIBUCIÓN POR SUBREGIÓN:
  - Suroeste: 10 estaciones
  - Occidente: 6 estaciones
  - Urabá: 5 estaciones
  - Nordeste: 4 estaciones
  - Bajo Cauca: 3 estaciones
  - Magdalena Medio: 3 estaciones
  - Norte: 2 estaciones
  - Oriente: 2 estaciones
  - Valle de Aburra: 1 estaciones

💧 TOP 10 ESTACIONES - ÚLTIMO DÍA METEOROLÓGICO:
  - sp_140: 60.20 mm
  - sp_133: 57.91 mm
  - sp_109: 54.36 mm
  - sp_134: 48.26 mm
  - sp_103: 44.45 mm
  - sp_143: 42.67 mm
  - sp_142: 34.04 mm
  - sp_101: 31.24 mm
  - sp_150: 30.23 mm
  - sp_141: 29.46 mm

⏰ ACUMULADOS PROMEDIO:
  - 6 horas: 0.00 mm
  - 24 horas: 14.91 mm
  - 72 horas: 23.11 mm

❌ ESTACIONES PROBLEMÁTICAS:
  - sp_108: 11 días sin datos
  - sp_147: 298 días sin datos

📄 Archivo CSV generado: metadatos_estaciones_precipitacion.csv
🌐 Aplicación Streamlit: streamlit_app.py
⚡ Para gráficas inte

In [28]:
# 5. Mapa de ubicación de todas las estaciones
fig_map = px.scatter_map(
    df_meta,
    lat="latitud",
    lon="longitud",
    hover_name="estacion",
    hover_data=["municipio", "subregion"],
    color="subregion",
    size_max=15,
    zoom=6,
    title="📍 Ubicación de todas las estaciones de precipitación"
)

fig_map.update_layout(
    mapbox_style="open-street-map",
    height=600
)
fig_map.show()

# 6. Serie de tiempo de una estación específica (ejemplo: la primera con datos)
if resultados:
    estacion_ejemplo = resultados[0]["estacion"]
    serie_ejemplo = resultados[0]["serie_120h"]
    
    if serie_ejemplo:
        df_serie_ejemplo = pd.DataFrame(serie_ejemplo)
        
        fig_serie = px.line(
            df_serie_ejemplo, 
            x="hora", 
            y="acumulado",
            title=f"Serie de tiempo 120h - Estación sp_{estacion_ejemplo}",
            labels={'acumulado': 'Acumulado (mm)', 'hora': 'Fecha y Hora'}
        )
        fig_serie.update_layout(xaxis_title="Hora", yaxis_title="Acumulado (mm)")
        fig_serie.show()
    else:
        print(f"No hay datos de serie temporal para la estación sp_{estacion_ejemplo}")

print("\n🎯 Estas gráficas funcionan perfectamente en el notebook!")
print("📱 Para una experiencia completa interactiva, usa el archivo streamlit_app.py")


🎯 Estas gráficas funcionan perfectamente en el notebook!
📱 Para una experiencia completa interactiva, usa el archivo streamlit_app.py


In [29]:
# Crear archivo separado para la aplicación Streamlit
streamlit_app_code = '''
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta
import requests
import warnings
from urllib3.exceptions import InsecureRequestWarning
import pytz
import time
import math

warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Configuración de la página
st.set_page_config(
    page_title="🌧️ Tablero de estaciones de precipitación",
    page_icon="🌧️",
    layout="wide"
)

@st.cache_data(ttl=3600)  # Cache por 1 hora
def cargar_datos():
    """Función para cargar y procesar todos los datos"""
    
    # Estaciones de precipitación (sp)
    sp_codes = ["101", "102", "103", "104", "106", "108", "109", "131", "132", "133", "134", "135", 
                "136", "137", "138", "139", "140", "141", "142", "143", "144", "145", "146", "147", 
                "149", "150", "151", "152", "154", "155", "156", "157","158","159", "160", "161", "162", "163"]

    def obtener_datos_estacion(code, calidad=1):
        page = 1
        datos = []
        while True:
            url = f"https://sigran.antioquia.gov.co/api/v1/estaciones/sp_{code}/precipitacion?calidad={calidad}&page={page}"
            response = requests.get(url, verify=False)
            if response.status_code != 200:
                break
            data = response.json()
            values = data.get("values", [])
            if not values:
                break
            datos.extend(values)
            page += 1
            # Paramos si ya tenemos más de 72 horas de datos
            fechas = [pd.to_datetime(d['fecha']) for d in datos]
            if fechas and (max(fechas) - min(fechas)).total_seconds() > 72 * 3600:
                break
        return datos

    def obtener_metadata_sp(code):
        url = f"https://sigran.antioquia.gov.co/api/v1/estaciones/sp_{code}/"
        resp = requests.get(url, verify=False)
        if resp.status_code == 200:
            d = resp.json()
            return {
                "estacion": code,
                "codigo": d.get("codigo"),
                "descripcion": d.get("descripcion"),
                "nombre_web": d.get("nombre_web"),
                "latitud": float(d.get("latitud", 0)),
                "longitud": float(d.get("longitud", 0)),
                "municipio": d.get("municipio"),
                "region": d.get("region")
            }
        else:
            return None

    def procesar_datos(datos, ahora=None):
        if not datos:
            return None

        df = pd.DataFrame(datos)
        df["fecha"] = pd.to_datetime(df["fecha"], utc=True)
        df["muestra"] = pd.to_numeric(df["muestra"], errors='coerce')

        ahora = ahora or datetime.utcnow().replace(tzinfo=pytz.UTC)

        acumulados = {
            "acum_6h": df[(df["fecha"] > ahora - timedelta(hours=6)) & (df["fecha"] <= ahora)]["muestra"].sum(),
            "acum_24h": df[(df["fecha"] > ahora - timedelta(hours=24)) & (df["fecha"] <= ahora)]["muestra"].sum(),
            "acum_72h": df[(df["fecha"] > ahora - timedelta(hours=72)) & (df["fecha"] <= ahora)]["muestra"].sum()
        }

        # Serie de 120 horas
        serie_120h = []
        for h in range(1, 121):
            t_ini = ahora - timedelta(hours=h)
            t_fin = ahora - timedelta(hours=h-1)
            val = df[(df["fecha"] > t_ini) & (df["fecha"] <= t_fin)]["muestra"].sum()
            serie_120h.append({"hora": t_ini, "acumulado": val})

        def acum_dias_meteorologicos(n, df):
            # Momento actual en UTC
            ahora = datetime.now(timezone.utc)
            # 7:00 AM UTC del día actual (inicio del día meteorológico actual)
            inicio_meteo = datetime.combine(ahora.date(), time(7, 0, tzinfo=timezone.utc))
            # Rango del día meteorológico: de (hace n días a las 7 AM) hasta (hoy a las 7 AM)
            fecha_inicio = inicio_meteo - timedelta(days=n)
            fecha_fin = inicio_meteo
            # Filtrar y sumar la columna 'muestra' en ese rango
            return df[(df["fecha"] > fecha_inicio) & (df["fecha"] <= fecha_fin)]["muestra"].sum()

        # Diccionario con los acumulados meteorológicos
        meteo = {
            "ultimo_dia_meteorologico": acum_dias_meteorologicos(1, df),
            "ultimos_7_dias_meteorologicos": acum_dias_meteorologicos(7, df),
            "ultimos_30_dias_meteorologicos": acum_dias_meteorologicos(30, df)
        }
        
        fecha_max = df["fecha"].max()
        dias_sin_datos = (ahora - fecha_max).days
        datos_recientes = int((ahora - fecha_max) <= timedelta(days=1))
        
        return {
            **acumulados,
            **meteo,
            "datos_recientes": datos_recientes,
            "dias_sin_datos": dias_sin_datos,
            "fecha_ultimo_dato": fecha_max, 
            "serie_120h": serie_120h
        }

    # Procesar datos
    progress_bar = st.progress(0)
    status_text = st.empty()
    
    resultados = []
    metadata = []
    
    total_estaciones = len(sp_codes)
    
    for i, code in enumerate(sp_codes):
        status_text.text(f'Procesando estación {code}... ({i+1}/{total_estaciones})')
        progress_bar.progress((i + 1) / total_estaciones)
        
        datos = obtener_datos_estacion(code)
        resumen = procesar_datos(datos)
        meta = obtener_metadata_sp(code)
        if resumen and meta:
            resumen["estacion"] = code
            meta.update(resumen)
            resultados.append(resumen)
            metadata.append(meta)
    
    progress_bar.empty()
    status_text.empty()
    
    df_meta = pd.DataFrame(metadata)
    
    # Aplicar correcciones de regiones
    correcciones = {
        'sp_163': 8,
        'sp_149': 3,
        'sp_151': 6,
        'sp_158': 6
    }
    for codigo, region_correcta in correcciones.items():
        df_meta.loc[df_meta['codigo'] == codigo, 'region'] = region_correcta
    
    # Procesar municipios y subregiones
    try:
        # Buscar el archivo en diferentes ubicaciones posibles
        import os
        possible_paths = [
            'Base de datos estaciones SAMA.xlsx',  # Directorio actual
            '/Users/sergiocamilogarzonperez/Projects/sama/pronosticos/Base de datos estaciones SAMA.xlsx',  # Ruta completa
            '../Base de datos estaciones SAMA.xlsx'  # Directorio padre
        ]
        
        excel_path = None
        for path in possible_paths:
            if os.path.exists(path):
                excel_path = path
                break
        
        if excel_path:
            df_excel = pd.read_excel(excel_path, usecols=[
                'GRUPO', 'MUNICIPIO', 'NOM_EST', 'COD_EST', 'TIPO', 'COMUN_PRIORIZ', 'CORRIENTE', 'LAT', 'LONG'
            ])
            df_excel = df_excel[['COD_EST', 'TIPO', 'GRUPO', 'MUNICIPIO', 'NOM_EST', 'COMUN_PRIORIZ', 'CORRIENTE', 'LAT', 'LONG']]
            df_excel['COD_EST'] = df_excel['COD_EST'].astype(str).str.strip().str.lower()
            
            df_meta = df_meta.rename(columns={'municipio': 'municipio_num'})
            df_municipio = df_excel[['COD_EST', 'MUNICIPIO']].rename(columns={
                'COD_EST': 'codigo',
                'MUNICIPIO': 'municipio'
            })
            df_meta = df_meta.merge(df_municipio, on='codigo', how='left')
            df_meta['municipio'] = df_meta['municipio'].str.capitalize()
            df_meta.loc[df_meta['codigo'] == 'sp_151', 'municipio'] = 'Sonson'
        else:
            st.warning("No se encontró el archivo Excel con datos de municipios. Se usarán datos básicos.")
            df_meta['municipio'] = 'Sin información'
    except Exception as e:
        st.warning(f"Error al cargar archivo Excel: {e}. Se usarán datos básicos.")
        df_meta['municipio'] = 'Sin información'
    
    # Mapear subregiones
    df_meta = df_meta.rename(columns={'region': 'subregion_num'})
    mapa_subregiones = {
        1: 'Valle de Aburra',
        2: 'Bajo Cauca',
        3: 'Magdalena Medio',
        4: 'Nordeste',
        5: 'Norte',
        6: 'Oriente',
        7: 'Occidente',
        8: 'Suroeste',
        9: 'Urabá'
    }
    df_meta['subregion'] = df_meta['subregion_num'].map(mapa_subregiones)
    
    # Procesar resultados
    df_resultado = pd.DataFrame([{k: v for k, v in r.items() if k != "serie_120h"} for r in resultados])
    df_resultado = df_resultado.sort_values(by=["datos_recientes", "fecha_ultimo_dato"], ascending=[False, False])
    
    df_pie = df_resultado.copy()
    df_pie['datos_recientes'] = df_pie['datos_recientes'].map({1: 'Reciente', 0: 'No reciente'})
    
    df_reciente = df_resultado[df_resultado["dias_sin_datos"] < 7].copy()
    df_reciente = df_reciente.sort_values(by='estacion', ascending=True)
    
    df_no_reciente = df_resultado[df_resultado["dias_sin_datos"] >= 7].copy()
    
    return df_meta, resultados, df_resultado, df_pie, df_reciente, df_no_reciente

# Título principal
st.title("🌧️ Tablero de estaciones de precipitación")

# Cargar datos
with st.spinner('Cargando datos de las estaciones...'):
    df_meta, resultados, df_resultado, df_pie, df_reciente, df_no_reciente = cargar_datos()

st.success(f'Datos cargados exitosamente. {len(df_reciente)} estaciones con datos recientes.')

# Sidebar con filtros
st.sidebar.header("Filtros")

# Filtro por subregión
subregiones = sorted(df_meta["subregion"].dropna().unique())
subregion_seleccionada = st.sidebar.selectbox(
    "Filtrar por subregión:",
    options=["Todas"] + subregiones,
    index=0
)

# Filtro por municipio (dinámico basado en subregión)
if subregion_seleccionada != "Todas":
    municipios = sorted(df_meta[df_meta["subregion"] == subregion_seleccionada]["municipio"].dropna().unique())
else:
    municipios = sorted(df_meta["municipio"].dropna().unique())

municipio_seleccionado = st.sidebar.selectbox(
    "Filtrar por municipio:",
    options=["Todos"] + municipios,
    index=0
)

# Filtro de estaciones (dinámico basado en filtros anteriores)
df_filtrado = df_meta.copy()
if subregion_seleccionada != "Todas":
    df_filtrado = df_filtrado[df_filtrado["subregion"] == subregion_seleccionada]
if municipio_seleccionado != "Todos":
    df_filtrado = df_filtrado[df_filtrado["municipio"] == municipio_seleccionado]

estaciones = sorted(df_filtrado["estacion"].unique())
if estaciones:
    estacion_seleccionada = st.sidebar.selectbox(
        "Selecciona estación:",
        options=[f"sp_{e}" for e in estaciones],
        index=0
    )
else:
    estacion_seleccionada = None
    st.sidebar.warning("No hay estaciones disponibles con los filtros seleccionados")

# Crear columnas para el layout
if estacion_seleccionada:
    col1, col2 = st.columns(2)
    
    # Serie de tiempo de 120 horas
    with col1:
        st.subheader("Serie de tiempo 120h")
        estacion_id = estacion_seleccionada.replace("sp_", "")
        serie = next((r["serie_120h"] for r in resultados if r["estacion"] == estacion_id), [])
        if serie:
            df_serie = pd.DataFrame(serie)
            fig_serie = px.line(df_serie, x="hora", y="acumulado", 
                              title=f"Serie 120h - {estacion_seleccionada}")
            fig_serie.update_layout(xaxis_title="Hora", yaxis_title="Acumulado (mm)")
            st.plotly_chart(fig_serie, use_container_width=True)
        else:
            st.info("No hay datos disponibles para esta estación")
    
    # Mapa de ubicación
    with col2:
        st.subheader("📍 Ubicación de la estación")
        estacion_id = estacion_seleccionada.replace("sp_", "")
        fila = df_meta[df_meta["estacion"] == estacion_id]
        if not fila.empty:
            fig_map = px.scatter_map(
                fila,
                lat="latitud",
                lon="longitud",
                hover_name="estacion",
                hover_data=["municipio", "subregion"],
                color_discrete_sequence=["red"],
                zoom=10,
                height=400
            )
            fig_map.update_layout(
                mapbox_style="carto-positron",
                margin={"r": 0, "t": 0, "l": 0, "b": 0},
                showlegend=False
            )
            st.plotly_chart(fig_map, use_container_width=True)
        else:
            st.info("Ubicación no disponible")

# Preparar datos para las tablas con filtros aplicados
df_tabla = df_reciente.merge(df_meta[["estacion", "subregion", "municipio"]], on="estacion", how="left")
if subregion_seleccionada != "Todas":
    df_tabla = df_tabla[df_tabla["subregion"] == subregion_seleccionada]
if municipio_seleccionado != "Todos":
    df_tabla = df_tabla[df_tabla["municipio"] == municipio_seleccionado]

# Tabla de acumulados recientes
st.subheader("Acumulados recientes por estación")
if not df_tabla.empty:
    df_acumulados = df_tabla[["estacion", "acum_6h", "acum_24h", "acum_72h"]].copy()
    df_acumulados["estacion"] = df_acumulados["estacion"].apply(lambda x: f"sp_{x}")
    df_acumulados = df_acumulados.round(3)
    df_acumulados.columns = ["Estación", "Acum. 6h", "Acum. 24h", "Acum. 72h"]
    
    col1, col2 = st.columns([3, 1])
    with col1:
        st.dataframe(df_acumulados, use_container_width=True)
    with col2:
        csv_acumulados = df_acumulados.to_csv(index=False)
        st.download_button(
            label="📥 Descargar CSV",
            data=csv_acumulados,
            file_name="acumulados_estaciones.csv",
            mime="text/csv"
        )
else:
    st.info("No hay estaciones con datos recientes para los filtros seleccionados")

# Tabla de acumulados meteorológicos
st.subheader("Acumulados meteorológicos por estación")
if not df_tabla.empty:
    df_meteo = df_tabla[["estacion", "ultimo_dia_meteorologico", "ultimos_7_dias_meteorologicos", "ultimos_30_dias_meteorologicos"]].copy()
    df_meteo["estacion"] = df_meteo["estacion"].apply(lambda x: f"sp_{x}")
    df_meteo = df_meteo.round(3)
    df_meteo.columns = ["Estación", "Último día", "Últimos 7 días", "Últimos 30 días"]
    
    col1, col2 = st.columns([3, 1])
    with col1:
        st.dataframe(df_meteo, use_container_width=True)
    with col2:
        csv_meteo = df_meteo.to_csv(index=False)
        st.download_button(
            label="📥 Descargar CSV",
            data=csv_meteo,
            file_name="acumulados_meteorologicos.csv",
            mime="text/csv"
        )

# Gráfico de acumulados meteorológicos del último día
st.subheader("Acumulado meteorológico del último día por estación")
if not df_tabla.empty:
    df_grafico = df_tabla.copy()
    df_grafico["estacion"] = df_grafico["estacion"].apply(lambda x: f"sp_{x}")
    
    fig_meteo = px.bar(
        df_grafico,
        x="estacion",
        y="ultimo_dia_meteorologico"
    )
    fig_meteo.update_layout(
        xaxis_title="Estación",
        yaxis_title="Acumulado (mm)",
        showlegend=False,
        xaxis_tickangle=45
    )
    st.plotly_chart(fig_meteo, use_container_width=True)

# Gráfico de estaciones sin datos
st.subheader("Estaciones sin datos por más de 7 días")
if not df_no_reciente.empty:
    df_sin_datos = df_no_reciente.copy()
    df_sin_datos["estacion"] = df_sin_datos["estacion"].apply(lambda x: f"sp_{x}")
    df_sin_datos = df_sin_datos.sort_values("dias_sin_datos", ascending=False)
    
    fig_sin_datos = px.bar(
        df_sin_datos,
        x="estacion", 
        y="dias_sin_datos"
    )
    fig_sin_datos.update_layout(
        xaxis_title="Estación",
        yaxis_title="Días sin datos",
        xaxis_tickangle=45
    )
    st.plotly_chart(fig_sin_datos, use_container_width=True)
else:
    st.success("🎉 Todas las estaciones tienen datos recientes!")

# Gráfico de disponibilidad de datos
st.subheader("Disponibilidad de datos recientes")
df_pie_chart = df_pie['datos_recientes'].value_counts()
total_estaciones = len(df_pie)

porcentajes = []
labels = []
values = []

for label in df_pie_chart.index:
    count = df_pie_chart[label]
    percentage = (count / total_estaciones) * 100
    porcentajes.append(f"{percentage:.1f}% ({count})")
    labels.append(label)
    values.append(count)

fig_disponibilidad = go.Figure(data=[
    go.Bar(
        x=labels,
        y=[(v/total_estaciones)*100 for v in values],
        text=porcentajes,
        textposition='inside',
        marker_color=['lightcoral' if 'No' in label else 'lightblue' for label in labels]
    )
])

fig_disponibilidad.update_layout(
    xaxis_title="Estado de los datos",
    yaxis_title="Porcentaje (%)",
    yaxis=dict(range=[0, 100]),
    height=400
)

st.plotly_chart(fig_disponibilidad, use_container_width=True)

# Información adicional en el sidebar
st.sidebar.markdown("---")
st.sidebar.markdown("### 📊 Estadísticas generales")
st.sidebar.metric("Total de estaciones", len(df_meta))
st.sidebar.metric("Con datos recientes", len(df_reciente))
st.sidebar.metric("Sin datos recientes", len(df_no_reciente))

# Información sobre actualización
st.sidebar.markdown("---")
st.sidebar.markdown("### ℹ️ Información")
st.sidebar.info("Los datos se actualizan automáticamente cada hora. La aplicación muestra datos de precipitación de las estaciones SAMA.")
'''

# Guardar el archivo
with open('/Users/sergiocamilogarzonperez/Downloads/streamlit_app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_app_code)

print("✅ Archivo 'streamlit_app.py' creado exitosamente en Downloads!")
print("Para ejecutar la aplicación, navega al directorio y usa: streamlit run streamlit_app.py")

✅ Archivo 'streamlit_app.py' creado exitosamente en Downloads!
Para ejecutar la aplicación, navega al directorio y usa: streamlit run streamlit_app.py


In [30]:
# Tiempo final
end = time.time() 

In [31]:
duracion_segundos = end - start
duracion_minutos = duracion_segundos / 60

# Redondear hacia arriba
duracion_minutos_redondeada = math.ceil(duracion_minutos)


print(f"Duración: {duracion_minutos_redondeada} minutos")

Duración: 1 minutos
